In [ ]:

def trainOp(model_relative_path: str = 'model', model_name: str = 'resnet_graphdef'):
    ### trainOp served as the main entry point (like main function for most of program languages)
    ### it takes several mandatory parameters to configure the path of saved/generated model file
    
    ### model_relative_path: A persistent volume by default will be mounted at `/home/jovyan` (we called application root here),
    ### to keep your state(i.e. files generated) in place.
    ### And by default, we want to keep our model generated in `model` relative to application root.
    ### However, in background job or hyperparameter tunning, this application root may CHANGED.
    ### So in order to perserve model, we use `model_relative_path` to discard variant application root caused.
    
    ### model_name: a model name will be generated as a file holder (i.e. folder) to all model files generated.
    ### In real scenario, the stucture of model file should depends on the inference infrastructure.
    ### and in our integration, we requires model should be structured in below:
    ### model/
    ###        $model_name/
    ###                    config.pbtxt
    ###                    labels.txt
    ###                    $version/
    ###                             model.savedmodel/
    ###                                              saved_models.pb
    import pathlib
    import os
    
    home = '/home/jovyan'
    output_model_dir = os.path.join(home, model_relative_path)
    model_dir = os.path.join(output_model_dir, model_name)
    
    # create an empty model dir
    pathlib.Path(model_dir).mkdir(parents=True, exist_ok=True)
    
    ## begin your code here
    ## ....
    ## end your code here
    
    
    ## use monitorCallback to let tintin to track your model performance, hence a comparison view can be used
    #class LossAndErrorPrintingCallback(keras.callbacks.Callback):
    #        def on_epoch_end(self, epoch, logs=None):
    #            print("Training-Accuracy={:7.6f}".format(logs["accuracy"]))
    #            print("Training-Loss={:7.6f}".format(logs["loss"]))
    #            print("Validation-Accuracy={:7.6f}".format(logs["val_accuracy"]))
    #            print("Validation-Loss={:7.6f}".format(logs["val_loss"]))
    #callbacks = [..., LossAndErrorPrintingCallback(), ...]
    #    # Run training, with or without data augmentation.
    #    if not data_augmentation:
    #        print('Not using data augmentation.')
    #        model.fit(x_train, y_train,
    #                  batch_size=batch_size,
    #                  epochs=epochs,
    #                  validation_data=(x_test, y_test),
    #                  shuffle=True,
    #                  callbacks=callbacks)
    
    return

In [ ]:
### if you want to debug the above tranOp function,
### uncomment below
### and remember to COMMENT it before you `BUILD` this pipeline through UI

# = trainOp('model', 'resnet_graphdef')

In [ ]:
### to access assets files, use tintin-sdk (requires python >=3.7)
### see tintin-sdk: https://github.com/footprintAI/tintin-sdk
#!pip install tintin-sdk --user

In [ ]:
## write all your requirements.txt here if possible
with open("requirements.txt", "w") as f:
    f.write("kfp==1.4.0\n")
    f.write("h5py<3.0.0\n")
    f.write("tintin-sdk\n")

!pip install -r requirements.txt --user --upgrade

In [ ]:
import kfp
import kfp.dsl as dsl
import kfp.components as comp
import kfp.compiler as compiler

In [ ]:
import os
pvcname = os.environ.get('TINTIN_SESSION_TEMPLATE_PVC_NAME')
generated_pipeline_zip_filename = os.environ.get('TINTIN_SESSION_TEMPLATE_GENERATED_PIPELINE_ZIP_FILENAME')
gpu_type_list_text = os.environ.get('TINTIN_SESSION_TEMPLATE_GPU_TYPE_LIST')
default_image = os.environ.get('TINTIN_SESSION_TEMPLATE_DEFAULT_IMAGE', 'footprintai/nvidia-tensorflow:19.12-tf1-py3')
mountPath = os.environ.get('TINTIN_SESSION_TEMPLATE_MOUNT_PATH', '/home/jovyan')



In [ ]:
trainComp = comp.func_to_container_op(trainOp, 
                                      base_image=default_image)

import kfp.dsl as dsl
@dsl.pipeline(
   name='Projectname pipeline',
   description='simple pipeline.'
)
def templated_pipeline_func(
):
    
    ### model relative path can NOT be nest path(e.g. a/b/c/d, it should be the first folder (e.g. model)
    model_relative_path = os.environ.get('TINTIN_SESSION_TEMPLATE_MODEL_RELATIVE_PATH', 'model')    
    model_name = os.environ.get('TINTIN_SESSION_TEMPLATE_MODEL_NAME', 'resnet_graphdef')
    ### if you want to customize $model_name, replace `my_customized_model_name` and uncomment below
    ### model_name = os.environ.get('TINTIN_SESSION_TEMPLATE_MODEL_NAME', 'my_customized_model_name')
    
    train_task = trainComp(model_relative_path, model_name)
    # add train_task default resources for cpu and memory, this value will be changed during runtime
    # to reflect your settings in UI
    train_task = train_task.add_resource_request('cpu', '1')
    train_task = train_task.add_resource_limit('cpu', '1')
    train_task = train_task.add_resource_request('memory', '4Gi')
    train_task = train_task.add_resource_limit('memory', '4Gi')
    
    # add annotation to reflect our configuration on `model_relative_path` and `model_name` to workflow itself.
    train_task = train_task.add_pod_annotation('tintin.footprint-ai.com/session-model-relative-path', model_relative_path)    
    train_task = train_task.add_pod_annotation('tintin.footprint-ai.com/session-model-name', model_name)
compiler.Compiler().compile(templated_pipeline_func, generated_pipeline_zip_filename)